**Set environment**

In [1]:
suppressWarnings(suppressMessages(source("../config/config_sing.R")))
show_env()

You are in Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out 
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC 
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc 
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log 


## Import data

In [2]:
fdiry = file.path(FD_RES, "results", "CRISPRi_FlowFISH", "coverage")
fname = "*_HCRFF_rAVG.log2FC_2.bed"
fglob = file.path(fdiry, fname)

fpaths = Sys.glob(fglob)
fpath_exclude = "Merge_HCRFF_rAVG.log2FC_2.bed"
fpaths = setdiff(fpaths, fpath_exclude)
fnames = basename(fpaths)

print(length(fpaths))
for (fname in fnames){cat(fname, "\n")}

[1] 20
CAPRIN1_HCRFF_rAVG.log2FC_2.bed 
CAT_HCRFF_rAVG.log2FC_2.bed 
CD164_HCRFF_rAVG.log2FC_2.bed 
ERP29_HCRFF_rAVG.log2FC_2.bed 
FADS1_HCRFF_rAVG.log2FC_2.bed 
FADS2_HCRFF_rAVG.log2FC_2.bed 
FADS3_HCRFF_rAVG.log2FC_2.bed 
FEN1_HCRFF_rAVG.log2FC_2.bed 
GATA1_HCRFF_rAVG.log2FC_2.bed 
HBE1_HCRFF_rAVG.log2FC_2.bed 
HBG1_HCRFF_rAVG.log2FC_2.bed 
HBG2_HCRFF_rAVG.log2FC_2.bed 
HBS1L_HCRFF_rAVG.log2FC_2.bed 
HDAC6_HCRFF_rAVG.log2FC_2.bed 
LMO2_HCRFF_rAVG.log2FC_2.bed 
MEF2C_HCRFF_rAVG.log2FC_2.bed 
MYB_HCRFF_rAVG.log2FC_2.bed 
MYC_HCRFF_rAVG.log2FC_2.bed 
NMU_HCRFF_rAVG.log2FC_2.bed 
PVT1_HCRFF_rAVG.log2FC_2.bed 


In [3]:
### get gene name
lst = str_split(fnames, pattern = "_")
vec = lapply(lst, function(vec){return(vec[1])})

### set columns
cnames = c("Chrom", "Start", "End", "Name", "Score")
ctypes = cols(
    "Chrom" = col_character(),
    "Start" = col_integer(),
    "End"   = col_integer(),
    "Name"  = col_character(),
    "Score" = col_double()
)

### read data
lst = lapply(fpaths, function(fpath){
    dat = read_tsv(fpath, col_names = cnames, col_types = ctypes)
    return(dat)
})
names(lst) = vec

### assign and show
lst_dat_crispri_hcrff = lst
print(length(lst))
for (txt in names(lst)){
    dat = lst[[txt]]
    cat(txt, nrow(dat), "\n")
}

[1] 20
CAPRIN1 57495 
CAT 57495 
CD164 57664 
ERP29 57381 
FADS1 9398 
FADS2 9398 
FADS3 9398 
FEN1 9398 
GATA1 13732 
HBE1 20485 
HBG1 20485 
HBG2 20485 
HBS1L 20485 
HDAC6 13732 
LMO2 57495 
MEF2C 57632 
MYB 20485 
MYC 49986 
NMU 43563 
PVT1 49986 


## Merge

In [4]:
lst = lst_dat_crispri_hcrff
dat = bind_rows(lst, .id = "Gene")
dat = dat %>% dplyr::select(Chrom, Start, End, Name, Score, Gene)

dat_crispri_hcrff_merge = dat
print(dim(dat))
head(dat)

[1] 656178      6


Chrom,Start,End,Name,Score,Gene
<chr>,<int>,<int>,<chr>,<dbl>,<chr>
chr10,100694991,100694992,id-1,0.436247,CAPRIN1
chr10,100695172,100695173,id-2,-1.057770,CAPRIN1
chr10,100695184,100695185,id-3,1.242010,CAPRIN1
chr10,100695209,100695210,id-4,-0.471250,CAPRIN1
chr10,100695328,100695329,id-5,-1.293510,CAPRIN1
chr10,100695346,100695347,id-6,0.928888,CAPRIN1


## Save

In [5]:
fdiry = file.path(FD_RES, "results", "CRISPRi_FlowFISH", "coverage")
fname = "Merge_HCRFF_rAVG.log2FC_2.bed.gz"
fpath = file.path(fdiry, fname)

dat = dat_crispri_hcrff_merge
write_tsv(dat, fpath, col_names=FALSE)